# Part 1 - CIFAR-10 Classification

## 1. Implement a shallow CNN

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, 3)
    self.conv2 = nn.Conv2d(32, 64, 3)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv3 = nn.Conv2d(64, 64, 3)
    self.conv4 = nn.Conv2d(64, 64, 3)
    self.fc = nn.Linear(6400, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)
    x = self.conv3(x)
    x = F.relu(x)
    x = self.conv4(x)
    x = F.relu(x)
    x = torch.flatten(x, 1)
    #x = nn.Flatten(x)
    x = self.fc(x)
    return x

print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net()
net = net.to(device)


#print(torch.cuda.get_device_name(0))
print(next(net.parameters()).device)

cpu


## 2. Use Pytorch Class torchvision.datasets.CIFAR10 to load the dataset

In [2]:
from torch.utils.data import random_split

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

testset, validationset = random_split(testset, [5000, 5000])

print(len(testset))
print(len(validationset))
# validationset.targets = validationset_labels
# validationset.data = validationsetdata

testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

validationloader = torch.utils.data.DataLoader(validationset, batch_size=32,
                                               shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:04<00:00, 34843071.92it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
5000
5000


## 3. Training, validation, and test

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.002)

train_loss =[]
validation_loss = []
validation_accuracy = []
train_accuracy = []

for epoch in range(60):

  running_loss = 0.0
  validation_running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  train_loss.append((epoch, running_loss / len(trainloader)))
  print('[%d] loss: %.3f' % (epoch + 1, running_loss / len(trainloader)))
  running_loss = 0.0
  if epoch % 3 == 0:
    test_correct = 0
    test_total = 0
    for data in testloader:

      inputs, labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = net(inputs)

      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == labels).sum().item()

    train_accuracy.append((epoch, (100 * test_correct / test_total)))
    print("Train accuracy at epoch %d: %.3f" % (epoch + 1, (100 * test_correct / test_total)))

    correct = 0
    total = 0
    with torch.no_grad():
      for data in validationloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        outputs = net(images)
        loss = criterion(outputs, labels)

        validation_running_loss += loss.item()
        validation_loss.append((epoch, validation_running_loss))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
      validation_accuracy.append((epoch, (100 * correct / total)))
      print("Validation accuracy at epoch %d: %.3f" % (epoch + 1, (100 * correct / total)))

print("Training done")

correct = 0
total = 0
with torch.no_grad():
  for data in testloader:
    images, labels = data
    images = images.to(device)
    labels = labels.to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

NameError: name 'torch' is not defined

## 4. Training Loss, validation loss, and validation accuracy

## 5. Test accuracy

## 6. Effects of Kernel size

## 7. ResNet18